<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/Preprocessing/notebooks/processed/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create function to preprocess as follows:


  1, Lowercasing letters

  2, Removing stop words

  3, Stemming words

  4, Tokenizing




In [51]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
import regex as re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [47]:
#Obtaining management discussion / git bash
!git clone https://github.com/sheldonkemper/bank_of_england.git
%ls
%cd bank_of_england/
%ls
%cd data
%ls
%cd processed/
%ls

#Defining qa_data
qa_data = pd.read_csv("qa_section.csv")


Cloning into 'bank_of_england'...
remote: Enumerating objects: 326, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 326 (delta 48), reused 32 (delta 24), pack-reused 232 (from 1)
Receiving objects: 100% (326/326), 3.23 MiB | 13.79 MiB/s, done.
Resolving deltas: 100% (129/129), done.
bank_of_england/  management_discussion.csv  qa_section.csv
/bank_of_england/bank_of_england/bank_of_england/data/bank_of_england/data/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england
data/  documents/  notebooks/  README.md  src/
/bank_of_england/bank_of_england/bank_of_england/data/bank_of_england/data/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank_of_england/data/processed/bank

In [48]:
qa_data.head()

,speaker,marker,job_title,utterance,filename,financial_quarter,call_date
0,Jeremy Barnum,A,"Chief Financial Officer, JPMorganChase","Yeah. I think the conventional wisdom on QT, a...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
1,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC","So, you'll stay around maybe for a few more ye...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
2,Mike Mayo,Q,"Analyst, Wells Fargo Securities LLC",All right. Thank you.,4q24-earnings-transcript.pdf,4Q24,2025-01-15
3,Operator,NaN,NaN,Thank you. Our next question comes from Jim Mi...,4q24-earnings-transcript.pdf,4Q24,2025-01-15
4,Jim Mitchell,Q,"Analyst, Seaport Global Securities LLC","Hey. Good morning. Maybe just on regulation, w...",4q24-earnings-transcript.pdf,4Q24,2025-01-15


In [35]:
#Checking the type of data
qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739 entries, 0 to 738
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   speaker            739 non-null    object
 1   marker             637 non-null    object
 2   job_title          636 non-null    object
 3   utterance          738 non-null    object
 4   filename           739 non-null    object
 5   financial_quarter  739 non-null    object
 6   call_date          739 non-null    object
dtypes: object(7)
memory usage: 40.5+ KB


In [58]:
#create function to preprocess data
def preprocessor (data, col):
  #Lower the lettercase
  data[col] = data[col].str.lower()
  #Remove stop words
  stop_words = set(stopwords.words("english"))
  data[col] = data[col].apply(lambda x: " ".join([word for word in x.split() if word not in (stop_words)]))
  return

In [61]:
print(qa_data["utterance"].dtype)

#preprocessor(qa_data, "utterance")

object


In [57]:
qa_data.head()

,speaker,marker,job_title,utterance,filename,financial_quarter,call_date
0,jeremy barnum,A,"Chief Financial Officer, JPMorganChase","yeah. i think the conventional wisdom on qt, a...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
1,mike mayo,Q,"Analyst, Wells Fargo Securities LLC","so, you'll stay around maybe for a few more ye...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
2,mike mayo,Q,"Analyst, Wells Fargo Securities LLC",all right. thank you.,4q24-earnings-transcript.pdf,4Q24,2025-01-15
3,operator,NaN,NaN,thank you. our next question comes from jim mi...,4q24-earnings-transcript.pdf,4Q24,2025-01-15
4,jim mitchell,Q,"Analyst, Seaport Global Securities LLC","hey. good morning. maybe just on regulation, w...",4q24-earnings-transcript.pdf,4Q24,2025-01-15
